In [7]:
"""Algebra utilities involved in the deduction procedure of fusion plasma physics formula.

Author: Wenyin Wei wenyin.wei@ipp.ac.cn

"""

def r_transform(a, from_coord, to_coord):
    """
    Args:
    from_coord='cartesian',to_coord='spherical'
    """
    from sympy import sin, cos, atan, sqrt, atan2
    if from_coord=='spherical' and to_coord=='cartesian':
        r, theta, phi = a[0], a[1], a[2]
        return _Array[(
            r*sin(theta)*cos(phi), 
            r*sin(theta)*sin(phi), 
            r*cos(theta))]
    if from_coord=='cartesian' and to_coord=='spherical':
        x, y, z = a[0], a[1], a[2]
        return _Array[(
            sqrt(x**2 + y**2 + z**2), 
            atan(sqrt(x**2+y**2)/z), 
            atan2(y, x))]
    elif from_coord=='cylindrical' and to_coord=='cartesian':
        rho, phi, z = a[0], a[1], a[2]
        return _Array[(
            rho*cos(phi), 
            rho*sin(phi), 
            z)]
    if from_coord == 'cartesian' and to_coord=='cylindrical':
        x, y, z = a[0], a[1], a[2] 
        return _Array[(
            sqrt(x**2+y**2), 
            atan2(y, x), 
            z)]
    else:
        raise NotImplementedError("The coordinate system transform, from {from_coord} to {to_coord}, for position vector r has not yet been prepared.")

def dot(a, b, metric=None): 
    if a.rank() !=1 or b.rank() !=1:
        raise ValueError("a, b should be vector, i.e. rank==1.")
    if a.shape != b.shape:
        raise ValueError("Unmatched Array shape. a, b shape should identical for dot operation.")
        
    if metric is None:
        res = 0
        for i in range(a.shape[0]):
            res += a[i]*b[i]
        return res
    else:
        raise ValueError('metric arg has not yet been prepared.')
        
def dot4r(a, b, coord='cartesian'):
    if coord!='cartesian':
        a = r_transform(a, from_coord=coord, to_coord='cartesian')
        b = r_transform(b, from_coord=coord, to_coord='cartesian')
    
    ans = dot(a, b, metric=None)
    
    if coord!='cartesian':
        return r_transform(ans, from_coord='cartesian', to_coord=coord)
    else:
        return ans
        
# def cross(a, b, metric=None):
#     if metric is None:
#         return _Array([a[1]*b[2]-a[2]*b[1], a[2]*b[0]-a[0]*b[2], a[0]*b[1]-a[1]*b[0]])
#     else:
#         raise ValueError('metric arg has not yet been prepared.')
        

def cross4r(a, b, coord='cartesian'):
    if coord!='cartesian':
        a = r_transform(a, from_coord=coord, to_coord='cartesian')
        b = r_transform(b, from_coord=coord, to_coord='cartesian')
    
    ans = cross(a, b, metric=None)
    
    if coord!='cartesian':
        return r_transform(ans, from_coord='cartesian', to_coord=coord)
    else:
        return ans

# def triple_prod(a, b, c, metric=None):
#     if a.rank() !=1 or b.rank() != 1 or c.rank() != 1:
#         raise ValueError("a, b, c should be vector, i.e. rank==1.")
#     if a.shape != (3) or b.shape != (3) or c.shape != (3):
#         raise ValueError("Unmatched Array shape. a, b, c should be vector in 3D.")
        
#     if metric is None:
#         return dot(cross(a, b, metric), c, metric)
#     else:
#         raise ValueError('metric arg has not yet been prepared.')

def norm(arr):
    from operator import add
    from functools import reduce
    from sympy import sqrt
    return sqrt(reduce(add, arr.applyfunc(lambda x: x**2)))


def diff4r(r, x, n, coord='cartesian'):
    if coord=='cartesian':
        return r.diff(x, n)
    elif coord=='spherical':
        raise NotImplementedError()
    elif coord=='cylindrical':
        raise NotImplementedError()
    return 


In [10]:
from sympy import symbols
r1, theta1, phi1, r2, theta2, phi2 = symbols('r_1, theta_1, phi_1, r_2, theta_2, phi_2', real=True, negative=False)
vr1 = (r1, theta1, phi1)
vr2 = (r2, theta2, phi2)
cross4r(vr1, vr2, coord='spherical')[0].simplify()

r_1*r_2*sqrt((sin(phi_1)*sin(theta_1)*cos(theta_2) - sin(phi_2)*sin(theta_2)*cos(theta_1))**2 + (sin(theta_1)*cos(phi_1)*cos(theta_2) - sin(theta_2)*cos(phi_2)*cos(theta_1))**2 + sin(theta_1)**2*sin(theta_2)**2*sin(phi_1 - phi_2)**2)

In [6]:
cross4r(vr1, vr2, coord='spherical')[1].simplify()

-atan(sqrt(2)*sqrt(-2*cos(2*theta_1 - 2*theta_2) - 2*cos(2*theta_1 + 2*theta_2) + cos(-phi_1 + phi_2 + 2*theta_1 + 2*theta_2) - cos(phi_1 - phi_2 - 2*theta_1 + 2*theta_2) - cos(phi_1 - phi_2 + 2*theta_1 - 2*theta_2) + cos(phi_1 - phi_2 + 2*theta_1 + 2*theta_2) + 4)/(4*sin(theta_1)*sin(theta_2)*sin(phi_1 - phi_2)))

In [7]:
cross4r(vr1, vr2, coord='spherical')[2].simplify()

atan2(r_1*r_2*(-sin(theta_1)*cos(phi_1)*cos(theta_2) + sin(theta_2)*cos(phi_2)*cos(theta_1)), r_1*r_2*(sin(phi_1)*sin(theta_1)*cos(theta_2) - sin(phi_2)*sin(theta_2)*cos(theta_1)))

### Cylindrical Coordinate

$$\vec{r}(\rho, \phi, z) = \rho\vec{e}_\rho(\phi) + z \vec{e}_z$$

Suppose $(\rho, \phi, z)$ has an argument $t$. (they can have multiple args, as the following deduction also holds.)

i.e. 

$$\rho = \rho(t)$$
$$\phi = \phi(t)$$
$$z = z(t)$$

$$\frac{\mathrm{d}\vec{r}(\rho, \phi, z)}{\mathrm{d}t} = \frac{\mathrm{d}\rho}{\mathrm{d}t}\vec{e}_\rho(\phi) + \rho\frac{\mathrm{d}\vec{e}_\rho}{\mathrm{d}t} + \frac{\mathrm{d}z}{\mathrm{d}t} \vec{e}_z$$
$$\frac{\mathrm{d}^n\vec{r}(\rho, \phi, z)}{\mathrm{d}t^n} = \sum_{k=0}^n C_n^k \frac{\mathrm{d}^k\rho}{\mathrm{d}t^k} \frac{\mathrm{d}^{n-k}}{\mathrm{d}t^{n-k}} \vec{e}_\rho(\phi) + \frac{\mathrm{d}^kz}{\mathrm{d}t^k} \vec{e}_z$$
$$\frac{\mathrm{d}^n\vec{r}(\rho, \phi, z)}{\mathrm{d}t^n} = \sum_{k=0}^n C_n^k \frac{\mathrm{d}^k\rho}{\mathrm{d}t^k} \frac{\mathrm{d}^{n-k}}{\mathrm{d}\phi^{n-k}} \vec{e}_\rho(\phi) \frac{\mathrm{d}^{n-k}\phi}{\mathrm{d}t^{n-k}} + \frac{\mathrm{d}^kz}{\mathrm{d}t^k} \vec{e}_z$$
$$\frac{\mathrm{d}^n\vec{r}(\rho, \phi, z)}{\mathrm{d}t^n} = \sum_{k=0}^n C_n^k \frac{\mathrm{d}^k\rho}{\mathrm{d}t^k} \frac{\mathrm{d}^{n-k}\phi}{\mathrm{d}t^{n-k}} \frac{\mathrm{d}^{n-k \pmod{4}}}{\mathrm{d}\phi^{n-k \pmod{4}}} \vec{e}_\rho(\phi) + \frac{\mathrm{d}^kz}{\mathrm{d}t^k} \vec{e}_z$$

Note that $\mathrm{d}\vec{e}_\rho(\phi)/ \mathrm{d}\phi = \vec{e}_\phi(\phi)$, $\mathrm{d}\vec{e}_\phi(\phi)/ \mathrm{d}\phi = -\vec{e}_\rho(\phi)$, therefore

$$\frac{\mathrm{d}\vec{e}_\rho(\phi)}{\mathrm{d}\phi} = \vec{e}_\phi(\phi)$$
$$\frac{\mathrm{d}^2\vec{e}_\rho(\phi)}{\mathrm{d}\phi^2} = -\vec{e}_\rho(\phi)$$
$$\frac{\mathrm{d}^3\vec{e}_\rho(\phi)}{\mathrm{d}\phi^3} = -\vec{e}_\phi(\phi)$$
$$\frac{\mathrm{d}^4\vec{e}_\rho(\phi)}{\mathrm{d}\phi^4} = \vec{e}_\rho(\phi)$$

### Metric Tensor

First basic form 
$$g_{ij} = \vec{r}_i \cdot \vec{r}_j$$

Second basic form 
$$\Omega_{ij} = \vec{r}_{ij}$$

### Christoffel symbol, also named as connection coefficients.

$$\Gamma^k_{ij} = \frac{1}{2} g^{km}(\frac{\partial g_{mj}}{\partial u^i} + \frac{\partial g_{im}}{\partial u^j} - \frac{\partial g_{ij}}{\partial u^m})$$

In [5]:
from sympy.abc import t, u, v
from sympy import cos, sin, pi

In [6]:
c = Curve([5*cos(t),5*sin(t), 3*t], (t, -2*pi, 2*pi))
c.curvature().simplify()

5/34

In [104]:
norm_c = c.param_norm()
print(norm_c)
norm_c._r

A curve = [5*cos(sqrt(34)*s/34), 5*sin(sqrt(34)*s/34), 3*sqrt(34)*s/34] in XYZ coordinate system, with s in (-2*sqrt(34)*pi, 2*sqrt(34)*pi).


[5*cos(sqrt(34)*s/34), 5*sin(sqrt(34)*s/34), 3*sqrt(34)*s/34]

In [13]:
from sympy import symbols, oo, sin, cos, pi
u, v = symbols('u, v', real=True)
R = 5
s = Surface([R*cos(u), R*sin(u), v], (u, 0, 2*pi), (v, -oo, oo))

In [8]:
str(c)

'A curve = [5*cos(t), 5*sin(t), 3*t] in XYZ coordinate system, with t in (-2*pi, 2*pi).'

In [8]:
import matplotlib.pyplot as plt

In [11]:
print(s

AttributeError: 'Surface' object has no attribute '_sys'

In [1]:
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
from sympy import lambdify, N

f_s = lambdify(s.uv(), s.r(), "numpy")

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Make data
u_limit = tuple([float(u) for u in s.u_limit()])
u = np.linspace(*u_limit, num=100)
v = np.linspace(0, 6.28, num=100)
ug, vg = np.meshgrid(u, v)
xg, yg, zg = f_s(ug, vg)

ax.plot_surface(xg, yg, zg, facecolors=cm.jet(zg/np.max(zg)), shade=True)

plt.show()

NameError: name 's' is not defined

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Equation of ring cyclide
# see https://en.wikipedia.org/wiki/Dupin_cyclide
import numpy as np
a, b, d = 1.32, 1., 0.8
c = a**2 - b**2
u, v = np.mgrid[0:2*np.pi:100j, 0:2*np.pi:100j]
x = (d * (c - a * np.cos(u) * np.cos(v)) + b**2 * np.cos(u)) / (a - c * np.cos(u) * np.cos(v))
y = b * np.sin(u) * (a - d*np.cos(v)) / (a - c * np.cos(u) * np.cos(v))
z = b * np.sin(v) * (c*np.cos(u) - d) / (a - c * np.cos(u) * np.cos(v))

fig = make_subplots(rows=1, cols=2,
                    specs=[[{'is_3d': True}, {'is_3d': True}]],
                    subplot_titles=['Color corresponds to z', 'Color corresponds to distance to origin'],
                    )

fig.add_trace(go.Surface(x=x, y=y, z=z, colorbar_x=-0.07), 1, 1)
fig.add_trace(go.Surface(x=x, y=y, z=z, surfacecolor=x**2 + y**2 + z**2), 1, 2)
fig.update_layout(title_text="Ring cyclide")
fig.show()